<a href="https://colab.research.google.com/github/mandresblanco/Salud_Chile/blob/main/Salud_chile_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introducción.**
 El síndrome metabólico (SM) es un grupo de cuadros que ponen en riesgo a una persona de desarrollar una enfermedad cardiaca y diabetes tipo 2. Estos cuadros son: (i) Hipertensión arterial; (ii) Glucosa alta en la sangre; (iii) Niveles sanguíneos elevados de triglicéridos; (iv) Bajos niveles sanguíneos de colesterol HDL; y (v) Exceso de grasa alrededor de 
la cintura (circunferencia de cintura). 



**Objetivo.** Caracterizar y determinar factores asociados a SM en Chile, usando datos de la 
Encuesta Nacional de Salud (ENS) 2009-2010. 


Diseño de la ENS. La encuesta consideró una muestra de 5412 personas, usando estratificación 
por regiones y según zona urbana – rural de cada región, el tamaño muestral por región se 
eligió en forma proporcional al número de habitantes de cada región. Al interior de cada región 
se seleccionaron al azar conglomerados correspondientes a las comunas y, dentro de las 
comunas, se eligieron al azar unidades censales (áreas definidas para el Censo de Población del 
INE). Finalmente, al interior de las unidades censales se seleccionaron hogares usando 
muestreo aleatorio simple. Se seleccionó una submuestra al azar de 4800 
individuos de la ENS2009.

In [4]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [7]:
%cd '/content/drive/My Drive/Colab Notebooks/Projects'
!ls

/content/drive/My Drive/Colab Notebooks/Projects
'Base ENS2009 Reducida.xlsx'		    country_vaccinations.csv
 country_profile_variables.csv		    kiva_country_profile_variables.csv
 country_vaccinations_by_manufacturer.csv   vaccines.csv


In [97]:
df = pd.read_excel('Base ENS2009 Reducida.xlsx')
df

,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN
1,2333,71,1,1.0,1.0,113.0,166.0,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0
2,403,61,1,1.0,1.0,114.9,162.0,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0
3,1059,68,2,1.0,2.0,92.0,161.0,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0
4,3310,46,2,1.0,2.0,89.4,150.0,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


IMC : Peso / Talla2

In [98]:
df.columns 
df['talla'] = df['talla']/100
df['IMC'] = df['peso'] / df['talla']**2
df


,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut,IMC
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN,NaN
1,2333,71,1,1.0,1.0,113.0,1.66,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0,41.007403
2,403,61,1,1.0,1.0,114.9,1.62,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0,43.781436
3,1059,68,2,1.0,2.0,92.0,1.61,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0,35.492458
4,3310,46,2,1.0,2.0,89.4,1.50,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0,39.733333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN,NaN
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ETARIO : EDAD en rangos: 

1 para ≤20,

2 para 21-40,

3 para 41-60, 

4 para >60 años


In [99]:
# df.reset_index()
# df = df.drop('oms', axis=1)
cond = [
               (df['edad']<=20),
               (df['edad']>20) & (df['edad']<41),
               (df['edad']>40) & (df['edad']<61),
               (df['edad']>60)]
opt = [1,2,3,4]
df['ETARIO'] = np.select(cond,opt)
df
# i=0



,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut,IMC,ETARIO
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN,NaN,4
1,2333,71,1,1.0,1.0,113.0,1.66,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0,41.007403,4
2,403,61,1,1.0,1.0,114.9,1.62,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0,43.781436,4
3,1059,68,2,1.0,2.0,92.0,1.61,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0,35.492458,4
4,3310,46,2,1.0,2.0,89.4,1.50,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0,39.733333,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN,NaN,4
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


SCORE_CV : Suma de factores de riesgo cardiovascular (CV): Edad>60, IMC>30, 
 pas130_pad85=1, Col_Total>200 y hdl40_50=1. 

el score va de 0 a 5

5 cuando cumple todos los factores de riesgo y 0 cuando no cumple ninguno

In [100]:

for i in range(len(df)):
  k=0
  if df.loc[i, 'edad'] > 60:
    k=k+1
  if df.loc[i, 'IMC'] >30.0:
    k=k+1
  if df.loc[i, 'pas130_pad85'] ==1.0:
    k=k+1
  if df.loc[i, 'col_total'] >200.0:
    k=k+1
  if df.loc[i, 'hdl40_50'] ==1.0:
    k=k+1
  # print(k)
  df.loc[i,'SCORE_CV'] = k
df


# cond = [
#                (df['edad']>60) & (df['IMC']>30) & (df['pas130_pad85']==1) & (df['Col_Total']>200) & (df['hdl40_50']==1),
#                (df['edad']>20) & (df['edad']<41),
#                (df['edad']>40) & (df['edad']<61),
#                (df['edad']>60)]
# opt = [1,2,3,4]
# df['SCORE_CV'] = np.select(cond,opt)
# df.head(50)

,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut,IMC,ETARIO,SCORE_CV
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN,NaN,4,1.0
1,2333,71,1,1.0,1.0,113.0,1.66,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0,41.007403,4,4.0
2,403,61,1,1.0,1.0,114.9,1.62,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0,43.781436,4,4.0
3,1059,68,2,1.0,2.0,92.0,1.61,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0,35.492458,4,4.0
4,3310,46,2,1.0,2.0,89.4,1.50,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0,39.733333,3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN,NaN,4,1.0
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0


CUAL_CV : Todas las posibles combinaciones en que se presentan los 5 factores 
 usados para construir SCORE_CV. 


In [110]:

for i in range(len(df)):
  if df.loc[i, 'edad'] > 60:
    k1 = '/edad '
  if df.loc[i, 'IMC'] >30.0:
    k1 = k1 + '/IMC '
  if df.loc[i, 'pas130_pad85'] ==1.0:
    k1 = k1 + '/pas130 '
  if df.loc[i, 'col_total'] >200.0:
    k1 = k1 + '/col '
  if df.loc[i, 'hdl40_50'] ==1.0:
    k1 = k1 + '/hd140 '
  df.loc[i,'CUAL_CV'] = k1
  # print(k1)
df

,ident,edad,sexo,zona,niveduc,peso,talla,circcintura,pas,pad,cintura88_83,glucosa100,triglic150,hdl40_50,pas130_pad85,num_criterios_sm,sm,col_total,col_hdl,col_ldl,triglic,glucosa,estado_nut,IMC,ETARIO,SCORE_CV,CUAL_CV
0,4536,91,2,1.0,1.0,NaN,NaN,NaN,119.333333,77.666667,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,110.0,NaN,NaN,4,1.0,/edad
1,2333,71,1,1.0,1.0,113.0,1.66,129.0,174.000000,99.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,124.0,28.0,80.0,79.0,108.0,4.0,41.007403,4,4.0,/edad /IMC /pas130 /hd140
2,403,61,1,1.0,1.0,114.9,1.62,128.0,164.666667,95.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,129.0,32.0,73.0,118.0,149.0,4.0,43.781436,4,4.0,/edad /IMC /pas130 /hd140
3,1059,68,2,1.0,2.0,92.0,1.61,120.0,131.333333,81.666667,1.0,1.0,0.0,1.0,1.0,4.0,1.0,178.0,45.0,108.0,129.0,112.0,4.0,35.492458,4,4.0,/edad /IMC /pas130 /hd140
4,3310,46,2,1.0,2.0,89.4,1.50,119.0,144.333333,87.333333,1.0,1.0,0.0,1.0,1.0,4.0,1.0,141.0,48.0,66.0,133.0,111.0,4.0,39.733333,3,3.0,/edad /IMC /pas130 /hd140 /IMC /pas130 /hd140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,5205,19,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,/edad
4796,5270,16,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,/edad
4797,5273,66,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117.0,NaN,NaN,4,1.0,/edad
4798,5389,19,1,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,/edad


In [111]:
dir_pandas='/content/drive/My Drive/Colab Notebooks/Projects/{}'
dir_pandas
df.to_csv(dir_pandas.format('Salud_Chile.csv'))

In [ ]:
  if df.loc[i, 'edad'] > 60 and df.loc[i, 'IMC'] >30.0 and df.loc[i, 'pas130_pad85'] ==1.0 and df.loc[i, 'col_total'] >200.0 and df.loc[i, 'hdl40_50'] ==1.0:
